<a href="https://colab.research.google.com/github/yoidea/DTMF-generator/blob/main/DTMF_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title **事前準備**

!pip install jaconv
from jaconv import z2h, hira2kata
from IPython.display import Audio
import numpy as np

In [2]:
# @title **初期設定**

SAMPLING_RATE = 48000 # @param{type: "number"}
SIGNAL_DURATION = 0.1 # @param{type: "number"}
PAUSE_DURATION = 0.1 # @param{type: "number"}

# ダイヤル番号の周波数
# https://elaws.e-gov.go.jp/document?lawid=360M50001000031#360M50001000031-Mpat_2
TONE_TABLE: dict[str, tuple[int, int]] = {
    "1": (697, 1209), "2": (697, 1336), "3": (697, 1477),
    "4": (770, 1209), "5": (770, 1336), "6": (770, 1477),
    "7": (852, 1209), "8": (852, 1336), "9": (852, 1477),
    "*": (941, 1209), "0": (941, 1336), "#": (941, 1477)
}

# フリーメッセージ番号一覧
# https://www.docomo.ne.jp/service/sms/usage/
CODE_TABLE: dict[str, str] = {
    "ｱ": "11", "ｲ": "12", "ｳ": "13", "ｴ": "14", "ｵ": "15",
    "ｶ": "21", "ｷ": "22", "ｸ": "23", "ｹ": "24", "ｺ": "25",
    "ｻ": "31", "ｼ": "32", "ｽ": "33", "ｾ": "34", "ｿ": "35",
    "ﾀ": "41", "ﾁ": "42", "ﾂ": "43", "ﾃ": "44", "ﾄ": "45",
    "ﾅ": "51", "ﾆ": "52", "ﾇ": "53", "ﾈ": "54", "ﾉ": "55",
    "ﾊ": "61", "ﾋ": "62", "ﾌ": "63", "ﾍ": "64", "ﾎ": "65",
    "ﾏ": "71", "ﾐ": "72", "ﾑ": "73", "ﾒ": "74", "ﾓ": "75",
    "ﾔ": "81", "(": "82", "ﾕ": "83", ")": "84", "ﾖ": "85",
    "ﾗ": "91", "ﾘ": "92", "ﾙ": "93", "ﾚ": "94", "ﾛ": "95",
    "ﾜ": "01", "ｦ": "02", "ﾝ": "03", "ﾞ": "04", "ﾟ": "05",
    "A": "16", "B": "17", "C": "18", "D": "19", "E": "10",
    "F": "26", "G": "27", "H": "28", "I": "29", "J": "20",
    "K": "36", "L": "37", "M": "38", "N": "39", "O": "30",
    "P": "46", "Q": "47", "R": "48", "S": "49", "T": "40",
    "U": "56", "V": "57", "W": "58", "X": "59", "Y": "50",
    "Z": "66", "?": "67", "!": "68", "ｰ": "69", "/": "60",
    "¥": "76", "&": "77", " ": "78", "☎": "79",
    "*": "86", "#": "87", "♥": "89",
    "1": "96", "2": "97", "3": "98", "4": "99", "5": "90",
    "6": "06", "7": "07", "8": "08", "9": "09", "0": "00"
}

In [7]:
# @title **出力波形の生成処理**

def generate_DTMF_wave(numbers: str):
    print(numbers)
    # ダイヤル番号から周波数に変換
    dial_tones = (TONE_TABLE[n] for n in numbers if n in TONE_TABLE)
    data = np.array([])
    t = np.linspace(0, SIGNAL_DURATION, int(SAMPLING_RATE*SIGNAL_DURATION))
    u = np.linspace(0, PAUSE_DURATION, int(SAMPLING_RATE*PAUSE_DURATION))

    # 波形を生成
    for tone in dial_tones:
        # 低群周波数
        low_tone = np.sin(2*np.pi*tone[0]*t)
        # 高群周波数
        high_tone = np.sin(2*np.pi*tone[1]*t)
        # 周波数合成
        data = np.append(data, low_tone+high_tone)
        # ポーズ挿入
        data = np.append(data, u*0)
    return data

In [ ]:
# @title **電話番号の送信**

DIAL_NUMBER = "090-0123-4567" # @param{type: "string"}

# DTMF信号に変換
data = generate_DTMF_wave(DIAL_NUMBER)
# 信号送出
Audio(data, rate=SAMPLING_RATE, autoplay=True)

In [ ]:
# @title **メッセージの送信**

MESSAGE = "テキストメッセージ" # @param{type: "string"}

# 半角カタカナに合わせて正規化
normalized_message = z2h(hira2kata(MESSAGE), kana=True, ascii=True, digit=True)
# メッセージ番号に変換
message_numbers = (CODE_TABLE[m] for m in normalized_message if m in CODE_TABLE)
# 仕様に合わせて先頭と末尾に番号追加
# https://www.docomo.ne.jp/service/sms/usage/
dial_number = "*2*2" + "".join(message_numbers) + "##"

# DTMF信号に変換
data = generate_DTMF_wave(dial_number)
# 信号送出
Audio(data, rate=SAMPLING_RATE, autoplay=True)